# Trabajo MC
## Redundancy Allocation Problem with Tabu Search

#### TO-DO LIST
- Implementar búsqueda tabú x
 - <span style="color:gray">(Tener en cuenta el tamaño máximo de la lista tabú {1}) </span>
- Estudiar heurística explicada en el paper
- Aplicar la heurística al algoritmo
- Desarrollar funciones de generación de problemas
- Desarrollar funciones de representación gráfica


> {1} Según el artículo, este tamaño máximo sera dinámico (hay que investigar más esto)


---
#### Dudas

- Paso 2 del algoritmo (¿cómo se hace el incremento aleatorio de la dirección?)
- mngli vs mnli
- U-function <span style="color:red;font-size:20px"> <-- ¡¡IMPORTANTE!! </span>


---

#### Comentarios

Un multi-state series-parallel system (MSS) es un sistema que tiene un número de componentes puestos en serie. Cada componente puede tener uno o más elementos del mismo tipo puestos en paralelo para aumentar la redundancia, y por tanto la fiabilidad del sistema, pero aumentando también su coste. El objetivo es minimizar el coste del diseño del sistema cumpliendo un requisito mínimo de fiabilidad.

Una vez seleccionada una version de un elemento para un componente, sólo se puede generar redundancia con elementos de esa versión.

- Solución inicial: Cada componente del sistema contiene sólo UN elemento de la VERSIÓN 1
- Incrementar la dirección de la solución actual aleatoriamente
- Aplicar búsqueda tabú al subespacio comprendido por los vecinos de la solución actual


La disponibilidad del sistema es el score que se va a considerar para evaluar la bondad de una solución, y es todo el rollo ese de las transformadas y nosecuanto que se aplica a las fiabilidades nominales (W) de cada elemento individual: Para cada componente, los rendimientos de los distintos elementos en paralelo se suman a través de la u-transformada esa rara. Para todos los componentes del sistema, el rendimiento es el mínimo de los rendimientos de los componentes, pero también a través de la u-transformada. (Miratelo Alvaro que por más que rasco no lo entiendo)

El espacio de estados -denotado como S- tiene (N - 2s + 1) posibles particiones (desde la mínima dirección posible, 2s; hasta la máxima, N).

A la hora de aplicar TS a un subespacio dado, es necesario extraer una estructura de **vecindario** para el mismo. Ésta estructura se obtiene a partir de todos los posibles movimientos únicos aplicados a la solución actual Y. Es necesario puntualizar que durante esta operación, la dirección (suma de las versiones de cada componente y redundancia de cada uno) de los vecinos se mantiene igual respecto a la de la solución original.

Un **movimiento** consiste en sumar y restar uno al número de versión y al de redundancia de cada componente de la solución, lo cual puede realizarse de tres maneras:

- Cambiando el número de redundancia de los componentes
- Cambiando el número de versión de los componentes
- Cambiando tanto el número de redundancia como el número de versión de los componentes

De entre todos los vecinos será necesario **descartar** los que sean tabú en el momento determinado y escoger aquel considerado como **mejor solución (Y’)**. Ésta solución deberá añadirse a la **lista tabú** (una lista limitada donde se almacenan las soluciones escogidas más recientes). <span style='color:#f1c232;'>El tamaño de ésta lista tabú tenemos que ver cómo lo calculamos.</span>  
Además, permitiremos que se pueda seleccionar una solución con una disponibilidad menor a la mínima requerida (A<sub>0</sub>), de tal manera que podamos atravesar la región no factible de soluciones. Para ello **añadiremos una penalización ponderada** a la función de coste para aquellas soluciones que violen las restricciones de disponibilidad.

Criterio de parada:

In [220]:
import random
import numpy as np
from scipy.stats import binom

In [295]:
# Esquema general de la búsqueda tabú
# https://en.wikipedia.org/wiki/Tabu_search
class TabuSearch():
    """
    s: número de componentes del sistema
    A_0: reliability mínimo requerido
    X_max: vector indicando el máximo Xi permitido (lista)
    J_max: vector indicando el máximo Ji permitido (lista)
    A: vector de disponibilidad de cada versión de los componente (lista de listas)
    C: vector de costes de cada versión de los componentes (lista de listas)
    W: vector de rendimientos nominales de cada versión de los componentes (lista de listas)    
    W_T: [(w_k,t_k),...] rendimiento w_k requerido durante t_k unidades de tiempo (lista de tuplas)
    q: penalización
    mnli: número máximo de iteraciones locales después de encontrar la mejor solución 
    mngi: número máximo de iteraciones globales después de encontrar la mejor solución
    discount_func: función de descuentos (función)
    """
    def __init__(self, s, A_0, X_max, J_max, A, C, W, W_T, q, mnli, mngi,discount_func=lambda x: 1):
        self.s = s
        self.A_0 = A_0
        self.X_max = X_max
        self.J_max = J_max
        self.A = A
        self.C = C
        self.W = W
        self.discount_func = discount_func
        self.W_T = W_T
        self.q = q
        self.mnli = mnli
        self.mngi = mngi
    
    def ts(self):
        stop_cond = self.mngi
        s0 = self.generate_s0()
        s_best = s0
        while stop_cond:
            stop_cond-=1
            s_cand = self.random_modification(s_best)
            s_sub_cand = self.ts_sub(s_cand)
            if(self.score(s_sub_cand)>self.score(s_best)):
                s_best=s_sub_cand
                stop_cond = self.mngi 
            
        return s_best
    
    def ts_sub(self, s0):
        s_best = s0
        stop_cond = self.mnli
        tabu_list = []
        tabu_list.append(s0)
        while stop_cond:
            stop_cond-=1  
            s_neighbors = self.neighborhood(s0, tabu_list)
            if(len(s_neighbors) == 0):
                continue
            s_cand = s_neighbors.pop()
            for neighbor in s_neighbors:
                if (not neighbor in tabu_list and self.score(neighbor)>self.score(s_cand)):
                    s_cand = neighbor
            if(self.score(s_cand)>self.score(s_best)):
                s_best=s_cand
                stop_cond = self.mnli
            tabu_list.append(s_cand)
            if(len(tabu_list) > self.mnli ):
                del tabu_list[0]
                          
        return s_best
    
    
    def neighborhood(self, solution, tabu_list):
        # Devuelve la lista de vecinos de solution
        def is_valid_neighbor(n, i1, i2):
            ''' Checkea si la modificación de Xi y/o de Ji de un vecino dado es válida (si estos valores se sitúan 
            entre 1 y X_max/J_max) y comprueba si no se encuentra ya en la lista tabu'''
            return (n[i1] in range(1, (self.X_max + self.J_max)[i1]+1) and
               n[i2] in range(1, (self.X_max + self.J_max)[i2]+1) and n not in tabu_list)

        def add_subtract_one_at_indexes(l,i1,i2):
            ''' Dados dos índices (i1,i2), se aplican las modificaciones (+1,-1) y (-1,+1) en los elementos
             correspondientes a las posiciones de dichos índices en la solución dada. Además, se checkea si
             dicha modificación es válida'''
            n1,n2 = l.copy(), l.copy()        
            n1[i1],n1[i2] = n1[i1]+1,n1[i2]-1
            n2[i1],n2[i2] = n2[i1]-1,n2[i2]+1            
            return [ n for n in [n1, n2] if is_valid_neighbor(n, i1, i2)]

        return [ n for i1 in range(len(solution)-1) 
                     for i2 in range(i1+1, len(solution)) 
                        for n in add_subtract_one_at_indexes(solution,i1,i2)]
    
    
    def score(self, solution):
        # Devuelve el score de solution de acuerdo con la heuristica del articulo        
        def extract_possible_m():
            def rec(index):
                if(index == self.s-1):                   
                    return list(range(1, solution[index]+1))
                else:
                    return [ [x, c] if( type(c) is not list) else [x]+c for x in range(1, solution[index]+1) for c in rec(index+1) ]
            return rec(0)        
        
        def m_performance(k_list):
            return min(k_list[i]*ij_performance(solution[self.s:][i],i) for i in range(len(k_list)))
        
        def ij_performance(version,component):            
            return self.W[component][version-1]        

        def delta_summ(k_interval,k_lists):
            m_sort = sorted(k_lists,key=lambda k: m_performance(k))
            return self.W_T[k_interval][1]*sum(delta(m) for m in m_sort if m_performance(m)>=self.W_T[k_interval][0])

        def delta(m):
            alpha = lambda k,x_i,a_ij: binom.pmf(k,x_i,a_ij)
            return np.prod([alpha(m[c],solution[:self.s][c],self.A[c][solution[self.s:][c]-1]) for c in range(len(m))])                
        
        def availability():
            possible_m = extract_possible_m()
            return ( (1/sum([ t_k for w_k, t_k in self.W_T ]))
                *sum([
                    delta_summ(k, possible_m)
                    for k, _ in enumerate(self.W_T)
                ]) )
        def cost():            
            return sum(solution[:self.s][c] * self.C[c][j-1] for c, j in enumerate(solution[self.s:]))
        
        return cost() + self.q*max(0, self.A_0 - availability())
    
    def generate_s0(self):
        s0 = [1,1]*self.s
        return s0
    
    def address(self, solution):
        return sum(solution)    
    
    def random_modification(self, solution):
        return solution


In [288]:
#%timeit -r 3 ns = neighborhood([ random.choice(range(10)) for _ in range(500)], [10]*500, [10]*500)

In [289]:
def str_to_problem(s, structure):
    lines = s.split("\n")
    
    A,C,W = [],[],[]
    for i,l in list(enumerate(lines)):
        if(len(l.strip().split(" "))>4):
            lines[i]=l.strip().split(" ")[1:]
        else:
            lines[i]=l.strip().split(" ")
    c0=0
    for n,c in list(enumerate(structure)):
        A.append([])
        C.append([])
        W.append([])
        for i in range(c):
            A[n].append(float(lines[i+c0][1]))
            C[n].append(float(lines[i+c0][2]))
            W[n].append(float(lines[i+c0][3]))
        c0+=c
    return A,C,W

In [290]:
''' Problem 3 '''
def problem_3(a_0):
    s = 4
    A_0 = [0.9, 0.96, 0.99] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*4
    J_max = [5, 4, 6, 5]
    A = [[0.970, 0.964, 0.980, 0.9869, 0.960],
         [0.967, 0.914, 0.960, 0.953],
         [0.959,0.970,0.959, 0.960, 0.970, 0.960],
         [0.989, 0.979, 0.980, 0.960, 0.980]]
    C = [[0.520, 0.620, 0.720, 0.890, 1.020],
         [0.516, 0.916, 0.967, 1.367],
         [0.214, 0.384, 0.534, 0.614, 0.783, 0.813],
         [0.683, 0.645, 0.697, 1.190, 1.260]]
    W = [[50, 80, 80, 100, 150],
         [20, 50, 50, 75],
         [60, 90, 180, 200, 200, 240],
         [25, 25, 30, 70, 70]]
    W_T = [(100, 20),(80, 30), (40, 50)]
    q = 5000
    mnli = 450
    mngi = 2500
    
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi)

In [291]:
p_3 = problem_3(0)
%timeit p_3.score([8,4,8,3,5,4,1,2])

325 ms ± 5.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [296]:
''' Problem 4 '''
def problem_4(a_0):
    s = 6
    A_0 = [0.975, 0.98, 0.99] # <- SOLO SELECCIONAR UNO 
    X_max = [10]*6
    J_max = [11, 8, 9, 4, 4, 6]
    
    A, C, W  = str_to_problem("1 1 0.932 1.590 27.3 \n 2 0.998 0.515 27.7 \n 3 0.983 0.225 49.8 \n 4 0.927 3.220 52.5 \n 5 0.959 4.020 62.0 \n 6 0.955 4.270 66.4 \n 7 0.984 3.670 84.6 \n 8 0.918 4.630 90.7 \n 9 0.939 1.010 97.0 \n 10 0.988 0.779 124 \n 11 0.984 3.130 129 \n 2 1 0.989 0.050 35.9 \n 2 0.923 1.290 44.7 \n 3 0.900 0.204 51.4 \n 4 0.946 2.220 63.2 \n 5 0.917 0.872 68.8 \n 6 0.962 1.830 81.8 \n 7 0.994 0.294 82.0 \n 8 0.984 2.810 115 \n 3 1 0.931 3.620 34.7 \n 2 0.950 0.475 41.0 \n 3 0.911 1.170 41.4 \n 4 0.956 0.793 43.6 \n 5 0.966 3.740 48.6 \n 6 0.992 4.590 59.6 \n 7 0.929 1.740 66.2 \n 8 0.968 1.720 91.9 \n 9 0.901 1.300 121 \n 4 1 0.915 2.490 25.1 \n 2 0.908 0.078 28.8 \n 3 0.928 1.370 50.2 \n 4 0.944 4.470 129 \n 5 1 0.908 1.550 34.9 \n 2 0.980 4.920 64.3 \n 3 0.964 2.650 108 \n 4 0.924 4.720 126 \n 6 1 0.965 3.220 24.8 \n 2 0.927 2.890 47.3 \n 3 0.986 3.410 58.8 \n 4 0.983 1.920 107 \n 5 0.991 4.510 120 \n 6 0.954 4.580 125", J_max)    
    W_T = [(100, 4203),(80, 788), (50, 1228), (20, 2536)]
    q = 400
    mnli = 35
    mngi = 100
    
    return TabuSearch(s, A_0[a_0], X_max, J_max, A, C, W, W_T, q, mnli, mngi)

In [304]:
p_4 = problem_4(2)
p_4.score([4,4,4,8,2,2,3,1,2,2,3,4])

12.764